In [ ]:
# mount drive to access data
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# change path HERE to reflect your folder structure
my_path = '/content/drive/MyDrive/suicide-project/data/'

In [ ]:
import pandas as pd
df_sn = pd.read_csv(my_path+'SN-only-FILTERED_PROCESSED_TEXT_CORPUS_v2.csv')

In [ ]:
df_sn.columns

Index(['post_id', 'lemmatized_body', 'keywords_matched', 'group_name',
       'monthyear'],
      dtype='object')

In [ ]:
import pandas as pd
from ast import literal_eval
df = pd.read_csv(my_path + 'PROCESSED_TEXT_CORPUS.csv',usecols = ['post_id','tokenized_body', 'date'],converters={'tokenized_body': literal_eval})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1337982 entries, 0 to 1337981
Data columns (total 3 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   post_id         1337982 non-null  int64 
 1   date            1337982 non-null  object
 2   tokenized_body  1337982 non-null  object
dtypes: int64(1), object(2)
memory usage: 30.6+ MB


In [ ]:
df_sn_text = df[df['post_id'].isin(df_sn['post_id'])]
df_sn = df_sn_text
del df_sn_text

In [ ]:
df_sn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62178 entries, 567 to 1337980
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   post_id         62178 non-null  int64 
 1   date            62178 non-null  object
 2   tokenized_body  62178 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.9+ MB


In [ ]:
df_sn['processed_body'] = df_sn['tokenized_body'].apply(lambda x: ' '.join(x))
df_sn['processed_body'].head(2)

<ipython-input-29-0c2010f1385f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sn['processed_body'] = df_sn['tokenized_body'].apply(lambda x: ' '.join(x))


567     nervous been planning and reading a lot for wh...
2410    excitedhappy knowing that ill be able to buy s...
Name: processed_body, dtype: object

In [ ]:
df_sn.drop(['tokenized_body'], axis=1)
del df_sn['tokenized_body']

In [ ]:
df_sn.reset_index(drop = True,inplace = True)

In [ ]:
df_sn.loc[11239]['processed_body']

'today i recieved my sn testing kit i am relieved waiting for sn which may take 47 days some changes in plan wont buy metoclopramide or anti acid till i get sn in my hands cause getting those is relatively easy will just need to go to my local medical shop i am thinking of buying sn from 2 sources and using the one with higher purity in the test but that may look fishy infront of my parents lets see did i mention i have been on no fap till i get my sn and everything just so i dont loose my focus and dont fuck anything up if i mess up anything i might end up damaging my body'

In [ ]:
sn_purchase_keywords = ['procure','procured','procures','source', 'sources','sells','sell', 'sold','seller','sellers','buy','buys',
                        'bought','get sn from','gets sn from','getting sn from','got sn from', 'purchased', 'purchase', 'purchases',
                        'order', 'ordered','orders']

In [ ]:
df_sn.loc[0,'processed_body']

'nervous been planning and reading a lot for when my sn and dmso get here by wednesday and im afraid of fucking up and dying a horrible death sn sounds like a really awful way to go if you screw up'

In [ ]:
df_sn_purchased = pd.DataFrame(columns = df_sn.columns)
for i in range(len(df_sn)):
  for x in sn_purchase_keywords:
    if x in df_sn.loc[i,'processed_body']:
      df_sn_purchased.loc[len(df_sn_purchased)] = df_sn.loc[i]

In [ ]:
len(df_sn_purchased)

24160

In [ ]:
df_sn_purchased['date'].head(2)

0     Oct 8, 2019
1    Mar 18, 2019
Name: date, dtype: object

In [ ]:
# find month + year associated with each post
df_sn_purchased['date_n'] = pd.to_datetime(df_sn_purchased['date'])    # converting string type to datetime object
list_year = [i.split(" ")[0][0:4] for i in list(df_sn_purchased['date_n'].astype(str))]
list_month = [i.split(" ")[0][5:7] for i in list(df_sn_purchased['date_n'].astype(str))]
df_sn_purchased['year'] = list_year
df_sn_purchased['month'] = list_month
df_sn_purchased.drop(['date_n'], axis=1)
del df_sn_purchased['date_n']
df_sn_purchased.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24160 entries, 0 to 24159
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   post_id         24160 non-null  object
 1   date            24160 non-null  object
 2   processed_body  24160 non-null  object
 3   year            24160 non-null  object
 4   month           24160 non-null  object
dtypes: object(5)
memory usage: 1.1+ MB


In [ ]:
df_sn_purch = df_sn_purchased
df_sn_purch.drop(['date','post_id','processed_body'], axis=1)
del df_sn_purch['date'], df_sn_purch['post_id'], df_sn_purch['processed_body']
df_sn_purch

,year,month
0,2019,10
1,2019,03
2,2020,01
3,2020,08
4,2021,02
...,...,...
24155,2021,01
24156,2021,01
24157,2021,01
24158,2021,01


In [ ]:
# save data for monthly purchase frequency visualization

df_sn_purch.to_csv(my_path+'sn_purchase_frequency_month_year.csv',index=False)